# Software Coaching for Python
# Week 5: Document Clustering & Topic Modeling - Homework

Instructor: Kang-Pyo Lee

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
path = "/content/gdrive/Shareddrives/Software_Coaching_Fall_2021"
my_folder = "HLP"     # *** REPLACE WITH YOUR FOLDER NAME ***
outcome_folder = f"{path}/{my_folder}/outcome"
classdata_folder = f"{path}/classdata"

Mounted at /content/gdrive


In [2]:
! pip install --user scikit-learn pyldavis

     |████████████████████████████████| 1.7 MB 5.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 60 kB/s 
     |████████████████████████████████| 11.3 MB 14.0 MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=a194b0a6bb50bb9ae9db76b8c329070b111afbb58540f81e0b23f3df4586f84e
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)

df = pd.read_csv(f"{classdata_folder}/timeline_NASA.csv", sep="\t")
df.text = df.text.astype(str)
df

,status_id,created_at,text,retweet_count,favorite_count,is_retweet,retweet_created_at
0,1354552906175934477,Wed Jan 27 22:12:33 +0000 2021,"It once held a lake. And what we're going to do, as soon as we get there, is to explore the rocks that were deposi… https://t.co/UW9wngvx4C",72,462,0,NaN
1,1354549784061865986,Wed Jan 27 22:00:09 +0000 2021,"I asked the team [...] to come up with something that would symbolize, to mark this challenge and thank in particu… https://t.co/QH235r9zjq",50,361,0,NaN
2,1354546326139265030,Wed Jan 27 21:46:25 +0000 2021,"Perseverance is the very first rover designed to seek signs of past microbial life, by collecting and caching rock… https://t.co/nn72OHRVv6",135,870,0,NaN
3,1354542309988327425,Wed Jan 27 21:30:27 +0000 2021,"🔴 LIVE: Tune in for a preview of the next Mars landing! On Feb. 18, our @NASAPersevere rover will arrive at the Red… https://t.co/rHRhIrzXWR",374,1430,0,NaN
4,1354540246436872195,Wed Jan 27 21:22:15 +0000 2021,RT @NASAMars: Don't forget to join us for today's briefing at 4:30pm ET (21:30 UTC): https://t.co/fGItMYWnFR Tag your questions #Countdown…,137,0,1,Wed Jan 27 20:18:56 +0000 2021
...,...,...,...,...,...,...,...
3195,1265370733930307585,Tue May 26 19:54:27 +0000 2020,RT @NASAKennedy: Looking for live views of the launch pad? We got you covered. 🚀👀Check out the rocket that will carry @AstroBehnken and @…,1197,0,1,Tue May 26 19:36:00 +0000 2020
3196,1265365848098816002,Tue May 26 19:35:02 +0000 2020,"For the first time since 2011, we’re sending American astronauts back to space, on an American rocket, from America… https://t.co/FsC1feO84c",622,2614,0,NaN
3197,1265327291468562432,Tue May 26 17:01:49 +0000 2020,LIVE: Want to go behind-the-scenes of #LaunchAmerica?Join our experts on @reddit to ask questions about the missi… https://t.co/p7iqJY4jyX,253,1942,0,NaN
3198,1265327258274840576,Tue May 26 17:01:41 +0000 2020,@brandonleblanc @SpaceX @AmyShiraTeitel See you online! https://t.co/FaQ7K75sBP,2,6,0,NaN


Check if there are any duplicates in the `text` column.

In [4]:
df.text.value_counts()

RT @NASAExhibit: 🤳Share your excitement about space exploration by posting a selfie on Twitter &amp; Instagram tagged with #NASAVirtualExhibit!…    2
Tonight's launch from @NASA_Wallops has been scrubbed. Stand by for updates. https://t.co/CmSYuve8c4                                                1
RT @Space_Station: The International Space Station orbited above Hurricane Zeta as it churned in the Gulf of Mexico just before 1pm ET toda…        1
RT @JimBridenstine: I’m looking forward to my upcoming virtual discussion with @SpaceForceCSO hosted by @mitchellstudies. On Tuesday at 9:3…        1
RT @jmorhard: .@NASA exists to help humankind, and we are doing our part to fight #COVID-19 using science &amp; technology. Here are the winner…    1
                                                                                                                                                   ..
LIVE NOW: Almost home! @Astro_SEAL and cosmonauts Anatoly Ivanishin and Ivan Vagner of @Roscosmos ha

Remove duplicates in the `text` column, if any. 

In [5]:
df = df.drop_duplicates(["text"], keep="first").copy()

In [6]:
df.text.value_counts()

Tonight's launch from @NASA_Wallops has been scrubbed. Stand by for updates. https://t.co/CmSYuve8c4                                                1
Everything is going exactly according to plan right now, it looks really good. - @OSIRISREx principal investigato… https://t.co/Cwm7kKevSa          1
RT @JimBridenstine: I’m looking forward to my upcoming virtual discussion with @SpaceForceCSO hosted by @mitchellstudies. On Tuesday at 9:3…        1
RT @jmorhard: .@NASA exists to help humankind, and we are doing our part to fight #COVID-19 using science &amp; technology. Here are the winner…    1
RT @Space_Station: .@AstroBehnken and @Astro_Doug are go for splashdown off the coast of Pensacola, Florida, today at 2:48pm ET aboard th…          1
                                                                                                                                                   ..
LIVE NOW: Almost home! @Astro_SEAL and cosmonauts Anatoly Ivanishin and Ivan Vagner of @Roscosmos ha

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3199 entries, 0 to 3199
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   status_id           3199 non-null   int64 
 1   created_at          3199 non-null   object
 2   text                3199 non-null   object
 3   retweet_count       3199 non-null   int64 
 4   favorite_count      3199 non-null   int64 
 5   is_retweet          3199 non-null   int64 
 6   retweet_created_at  815 non-null    object
dtypes: int64(4), object(3)
memory usage: 199.9+ KB


## Preparing Data for Modeling

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'averaged_perceptron_tagger', 'stopwords'])

import string 
global_stopwords = stopwords.words("english")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Define your own local stopwords. 

In [9]:
local_stopwords = [c for c in string.punctuation] +\
                  ['‘', '’', '—', '…', '“', '”', "'re", "'s", '...', 'a…', 't…', 'https', "n't", 'rt']

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, norm="l2", stop_words=global_stopwords+local_stopwords, max_df=0.7)
X = vectorizer.fit_transform(df.text)

In [11]:
X.shape

(3199, 8679)

## K-Means Clustering

### Step 1. Choose the number of clusters

Set the number of clusters you would like to get. 

In [12]:
# Your answer here
k = 5

### Step 2. Initialize a model object for k-means clustering

In [13]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=k, random_state=0)
kmeans

KMeans(n_clusters=5, random_state=0)

### Step 3. Fit the model using the input data

In [14]:
%time kmeans.fit(X)

CPU times: user 1.19 s, sys: 145 ms, total: 1.34 s
Wall time: 749 ms


KMeans(n_clusters=5, random_state=0)

### Step 4. Examine the clustering outcome

In [15]:
kmeans.labels_

array([0, 0, 1, ..., 0, 0, 4], dtype=int32)

In [16]:
df["cluster"] = kmeans.labels_

In [17]:
df[["text", "cluster"]]

,text,cluster
0,"It once held a lake. And what we're going to do, as soon as we get there, is to explore the rocks that were deposi… https://t.co/UW9wngvx4C",0
1,"I asked the team [...] to come up with something that would symbolize, to mark this challenge and thank in particu… https://t.co/QH235r9zjq",0
2,"Perseverance is the very first rover designed to seek signs of past microbial life, by collecting and caching rock… https://t.co/nn72OHRVv6",1
3,"🔴 LIVE: Tune in for a preview of the next Mars landing! On Feb. 18, our @NASAPersevere rover will arrive at the Red… https://t.co/rHRhIrzXWR",1
4,RT @NASAMars: Don't forget to join us for today's briefing at 4:30pm ET (21:30 UTC): https://t.co/fGItMYWnFR Tag your questions #Countdown…,0
...,...,...
3195,RT @NASAKennedy: Looking for live views of the launch pad? We got you covered. 🚀👀Check out the rocket that will carry @AstroBehnken and @…,0
3196,"For the first time since 2011, we’re sending American astronauts back to space, on an American rocket, from America… https://t.co/FsC1feO84c",0
3197,LIVE: Want to go behind-the-scenes of #LaunchAmerica?Join our experts on @reddit to ask questions about the missi… https://t.co/p7iqJY4jyX,0
3198,@brandonleblanc @SpaceX @AmyShiraTeitel See you online! https://t.co/FaQ7K75sBP,0


Check the number of values for each cluster. 

In [18]:
df.cluster.value_counts()

0    1993
4     454
3     403
1     219
2     130
Name: cluster, dtype: int64

Check 10 random texts from the largest cluster. 

In [19]:
df[df.cluster == 1].sample(10, random_state=0)[["text", "cluster"]]

,text,cluster
2299,We’re going to Mars! Our @NASAPersevere rover will soon launch to Mars where it will search for signs of ancient li… https://t.co/R8S7Vb7lIG,1
2120,🚀 We have LIFTOFF to Mars! The @ulalaunch Atlas V takes flight with our @NASAPersevere rover. The #CountdownToMars… https://t.co/wHgOvPrWlU,1
2115,"@jonnybuckIand @NASAPersevere We like that you like it! Here, have another one. #CountdownToMars",1
2342,"What will it take for future human missions to Mars to become a reality?From @NASAPersevere to #Artemis, here are… https://t.co/oeYEBr1enV",1
2344,"Right now, we're doing these robotic missions to Mars, so that when our humans do go to Mars... we're going to be… https://t.co/aobr1ETqzI",1
2270,Are you joining @NASAPersevere rover on its search for signs of ancient life on Mars? Here are seven things you nee… https://t.co/uIjKYQAgYE,1
2096,Signal confirmed! @NASASCaN has begun to receive a signal from the spacecraft carrying @NASAPersevere to the Red Pl… https://t.co/UDOdXIyuoq,1
2300,LIVE NOW: Could ancient Mars have supported life? @NASAPersevere will explore the Red Planet to collect rock and so… https://t.co/LBoxksAVwx,1
3035,Traveling to Mars this summer aboard @NASAPersevere: SHERLOC = an instrument to search for sand-grain size clues… https://t.co/HcCvDDFo1Q,1
2243,"Our @NASAPersevere rover is GO for launch on July 30. Today, get the latest on our #CountdownToMars (all times ET):… https://t.co/FbnEl9Ks3k",1


Check 10 random texts from the second largest cluster. 

In [20]:
df[df.cluster == 2].sample(10, random_state=0)[["text", "cluster"]]

,text,cluster
266,"On Jan. 6, @NorthropGrumman's #Cygnus spacecraft named in memory of Kalpana Chawla — the 1st female astronaut of In… https://t.co/ZulEZtlZgt",2
1179,"RT @NASASun: Today, @OSIRISREx attempts to collect a sample of asteroid Bennu. Studying this sample will help scientists learn about the fo…",2
1361,"After tonight's perfect launch and ride to orbit, the @NorthropGrumman Cygnus cargo spacecraft will arrive at the… https://t.co/9xi2qMbqNy",2
1407,"On Oct. 1, @NorthropGrumman's resupply mission to @Space_Station is set to lift off at 9:38pm ET from @NASA_Wallops… https://t.co/9VQEVlIUjI",2
1368,"The @NorthropGrumman Antares rocket and Cygnus cargo spacecraft, named the S.S. Kalpana Chawla, are go for launch… https://t.co/ynS8oBXFHR",2
1122,RT @JimBridenstine: Congratulations to the entire @OSIRISREx team and all of @NASA’s partners on this mission! We are on the way to returni…,2
1144,"🔄 HAPPENING NOW: Above asteroid Bennu, our @OSIRISREx spacecraft is rotating so that its robotic arm is pointed tow… https://t.co/kanFMCyWa3",2
1126,We're going in. Touchdown declared. Sampling is in progress. @OSIRISREx touched the surface of asteroid Bennu… https://t.co/2O4MV2Er9q,2
1184,"We're finally ready to go for TAG. Kenneth Getzandanner, flight dynamics manager for @OSIRISREx, explains how the… https://t.co/2XbfwZ71fQ",2
1103,RT @OSIRISREx: How it started: How it’s going: https://t.co/xT7hLo7SCl,2


In [21]:
import nltk
df["words"] = df.text.apply(lambda x: nltk.word_tokenize(x))
df["tagged_words"] = df.words.apply(lambda x: nltk.pos_tag(x))

from collections import Counter

def get_counter(dataframe, stopwords=[]):
    counter = Counter()
    
    for l in dataframe.tagged_words:
        word_set = set()

        for t in l:
            word = t[0].lower()
            tag = t[1]

            if word not in stopwords:
                word_set.add(word)
            
        counter.update(word_set)
        
    return counter

Check the top-30 most common keywords in the largest cluster. (Replace the cluster number as needed.)

In [22]:
counter0 = get_counter(df[df.cluster == 1], global_stopwords+local_stopwords)
counter0.most_common(30)

[('nasapersevere', 171),
 ('mars', 115),
 ('rover', 63),
 ('launch', 56),
 ('countdowntomars', 36),
 ('planet', 29),
 ('red', 28),
 ('mission', 24),
 ('july', 18),
 ('ulalaunch', 17),
 ('perseverance', 16),
 ('nasa', 16),
 ('first', 15),
 ('live', 14),
 ('next', 14),
 ('ready', 14),
 ('30', 14),
 ('today', 14),
 ('🚀', 13),
 ('go', 13),
 ('journey', 13),
 ('way', 12),
 ('jimbridenstine', 12),
 ('signs', 11),
 ('life', 11),
 ('us', 11),
 ('get', 11),
 ('going', 11),
 ('days', 10),
 ('helicopter', 10)]

Check the top-30 most common keywords in the second largest cluster. (Replace the cluster number as needed.)

In [23]:
counter4 = get_counter(df[df.cluster == 2], global_stopwords+local_stopwords)
counter4.most_common(30)

[('osirisrex', 86),
 ('spacecraft', 51),
 ('asteroid', 50),
 ('bennu', 45),
 ('northropgrumman', 33),
 ('sample', 32),
 ('cygnus', 31),
 ('mission', 26),
 ('space_station', 16),
 ('tag', 15),
 ('week', 14),
 ('launch', 14),
 ('oct.', 14),
 ('cargo', 13),
 ('amp', 13),
 ('20', 12),
 ('live', 12),
 ('today', 12),
 ('et', 10),
 ('surface', 10),
 ('next', 9),
 ('collect', 9),
 ('nasa_wallops', 8),
 ('lockheedmartin', 8),
 ('back', 7),
 ('resupply', 7),
 ('set', 7),
 ('named', 7),
 ('experts', 7),
 ('go', 7)]

## LDA Topic Modeling

### Step 1. Choose the number of topics

Set the number of topics you would like to get. 

In [24]:
# Your answer here
num_topics = 5

### Step 2. Initialize a model object for LDA topic modeling

In [25]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda = LDA(n_components=num_topics, random_state=0)
lda

LatentDirichletAllocation(n_components=5, random_state=0)

### Step 3. Fit the model using the input data

In [26]:
%time lda.fit(X)

CPU times: user 5.23 s, sys: 58.1 ms, total: 5.29 s
Wall time: 5.29 s


LatentDirichletAllocation(n_components=5, random_state=0)

### Step 4. Examine the output of topic modeling

In [27]:
def show_topics(model, feature_names, num_top_words):
    for topic_idx, topic_scores in enumerate(model.components_):
        print("***Topic {}:".format(topic_idx))
        print(" + ".join(["{:.2f} * {}".format(topic_scores[i], feature_names[i]) for i in topic_scores.argsort()[::-1][:num_top_words]]))
        print()

In [28]:
show_topics(lda, vectorizer.get_feature_names(), 10)

***Topic 0:
14.08 * space_station + 11.37 * crew + 9.76 * northropgrumman + 9.63 * spacex + 8.57 * dragon + 8.39 * spacecraft + 8.31 * us + 8.05 * space + 7.92 * mars + 7.81 * cygnus

***Topic 1:
17.48 * launch + 13.99 * nasa + 13.97 * et + 13.42 * today + 13.41 * sentinel + 12.78 * freilich + 12.70 * michael + 12.56 * launchamerica + 12.42 * live + 11.26 * space_station

***Topic 2:
16.86 * spacex + 15.44 * space_station + 12.03 * astrobehnken + 11.74 * launch + 11.23 * crew + 11.12 * amp + 9.48 * spacecraft + 9.36 * astro_doug + 8.95 * dragon + 8.81 * new

***Topic 3:
19.30 * space_station + 16.42 * astrobehnken + 15.91 * spacex + 15.71 * today + 14.10 * live + 13.89 * nasa + 12.74 * nasapersevere + 12.70 * launch + 12.53 * astronauts + 12.34 * space

***Topic 4:
13.13 * crew + 12.91 * nasa + 11.44 * space_station + 11.09 * mission + 10.82 * spacex + 9.38 * jimbridenstine + 9.08 * moon + 8.13 * launchamerica + 8.06 * astrobehnken + 7.83 * live



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Topic Model Visualization

In [36]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

ModuleNotFoundError: ignored

In [ ]:
pyLDAvis.sklearn.prepare(lda, X, vectorizer)